# Research Notebook for converting ToMi dataset to T4D

In [1]:
!python -m spacy download en_core_web_trf

     ---------------------------------------- 0.0/12.8 MB ? eta -:--:--
     ---------------------------------------- 0.0/12.8 MB ? eta -:--:--
     ---------------------------------------- 0.0/12.8 MB ? eta -:--:--
      --------------------------------------- 0.3/12.8 MB ? eta -:--:--
      --------------------------------------- 0.3/12.8 MB ? eta -:--:--
      --------------------------------------- 0.3/12.8 MB ? eta -:--:--
     - ------------------------------------- 0.5/12.8 MB 540.5 kB/s eta 0:00:23
     - ------------------------------------- 0.5/12.8 MB 540.5 kB/s eta 0:00:23
     -- ------------------------------------ 0.8/12.8 MB 588.4 kB/s eta 0:00:21
     -- ------------------------------------ 0.8/12.8 MB 588.4 kB/s eta 0:00:21
     --- ----------------------------------- 1.0/12.8 MB 613.4 kB/s eta 0:00:20
     --- ----------------------------------- 1.3/12.8 MB 664.1 kB/s eta 0:00:18
     --- ----------------------------------- 1.3/12.8 MB 664.1 kB/s eta 0:00:18
     ---

In [1]:
from datasets import load_dataset
from pyprojroot import here
import spacy

In [2]:
dataset = str(here("t4d/data/ToMi.csv"))
data = load_dataset("csv", data_files=dataset)

In [3]:
nlp = spacy.load("en_core_web_trf")

C:\Users\sachi\miniconda3\envs\t4d\lib\site-packages\thinc\shims\pytorch.py:253: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model.load_state_dict(torch.load(filelike, map

In [35]:
total = 0
for instance in data["train"]:
    doc = nlp(instance["question"])

    if len(doc.ents) == 1:
        total += 1

total

539

In [47]:
total = 0
for instance in data["train"]:
    if "exited" in instance["story"]:
        total += 1

total

1861

In [69]:
total = 0
for instance in data["train"]:
    if "moved the" in instance["story"]:
        total += 1

total

1861

In [114]:
total = 0

import re
pattern = r'first_order_[01]_no_tom'

for instance in data["train"]:
    if re.match(pattern, instance["qTypeRaw"]) :
        total += 1

total

389

In [118]:
total = 0

for instance in data["train"]:
    if instance["sType"] == "false_belief":
        total += 1

total

568

In [9]:
temp_subset = data["train"].filter(lambda x: x["qTypeTomOrNot"] == True)
temp_subset

Dataset({
    features: ['story', 'question', 'answer', 'i', 'dataGenSeq', 'sType', 'qTypeRaw', 'qTypeTomOrNot', 'qOrder', 'storyHasToM', 'answerMem', 'answerReal', 'qToMandOmniReader', 'answerMemOrReal', 'cands', 'correct', 'falseTrueBelief', 'factVsMind'],
    num_rows: 564
})

In [13]:
total = 0

for i, instance in enumerate(temp_subset):
    cs = nlp(instance["question"]).ents
    print(i, end=" ")
    if not cs:
        print(cs)
        print(instance)
        break

0 1 2 3 4 5 6 7 8 9 10 11 12 13 14 15 16 17 18 19 20 21 22 23 24 25 26 27 28 29 30 31 32 33 34 35 36 37 38 39 40 41 42 43 44 45 46 47 48 49 50 51 52 53 54 55 56 57 58 59 60 61 62 63 64 65 66 67 68 69 70 71 72 73 74 75 76 77 78 79 80 81 82 83 84 85 86 87 88 89 90 91 92 93 94 95 96 97 98 99 100 101 102 103 104 105 106 107 108 109 110 111 112 113 114 115 116 117 118 119 120 121 122 123 124 125 126 127 128 129 130 131 132 133 134 135 136 137 138 139 140 141 142 143 144 145 146 147 148 149 150 151 152 153 154 155 156 157 158 159 160 161 162 163 164 165 166 167 168 169 170 171 172 173 174 175 176 177 178 179 180 181 182 183 184 185 186 187 188 189 190 191 192 193 194 195 196 197 198 199 200 201 202 203 204 205 206 207 208 209 210 211 212 213 214 215 216 217 218 219 220 221 222 223 224 225 226 227 228 229 230 231 232 233 234 235 236 237 238 239 240 241 242 243 244 245 246 247 248 249 250 251 252 253 254 255 256 257 258 259 260 261 262 263 264 265 266 267 268 269 270 271 272 273 274 275 276 27

In [15]:
def find_characters(text):
    characters = []
    actions = ["entered", "moved", "exited"]
    
    words = text.split()

    for action in actions:
        indices = [index for index, word in enumerate(words) if word == action]

        for index in indices:
            character = words[index - 1]
            if character not in characters:
                characters.append(character)

    return characters

def find_character_who_moved(text):
    words = text.split()

    index = words.index("moved")

    return words[index - 1]

def find_moved_object(text):
    words = text.split()

    index = words.index("moved")

    return words[index + 2]

story = data["train"][4]["story"]
print(find_characters(story))
print(find_character_who_moved(story))
print(find_moved_object(story))

['Ella', 'Carter']
Carter
lemon


In [57]:
import random

instance = temp_subset[400]

characters = find_characters(instance["story"])
character_who_moved = find_character_who_moved(instance["story"])
moved_object = find_moved_object(instance["story"])

for c in nlp(instance["question"]).ents:
    if c.text != character_who_moved:
        answer = c.text

distracting_characters = [c for c in characters if c not in [answer, character_who_moved]]

cs = random.sample([character_who_moved, answer], 2)

append_statement = "{} and {} plan to use the {} soon.".format(*cs, moved_object)

question = "Who among the individuals would benefit from receiving helpful information?\n"
for i, c in enumerate(random.sample(characters, len(characters))):
    question += f"{chr(65 + i)}. {c}\n"
question += f"{chr(65 + len(characters))}. None of the above"

print(append_statement)
print(question)

Carter and Abigail plan to use the belt soon.
Who among the individuals would benefit from receiving helpful information?
A. Carter
B. Abigail
C. None of the above


In [26]:
instance

{'story': 'Abigail entered the hallway. Carter entered the hallway. The belt is in the suitcase. Carter exited the hallway. Abigail moved the belt to the bottle. Abigail exited the hallway. Carter entered the playroom.',
 'question': 'Where will Carter look for the belt?',
 'answer': 'suitcase',
 'i': 1,
 'dataGenSeq': 'enter_agent_0,enter_agent_1,agent_1_exits,agent_0_moves_obj,agent_0_exits,agent_1_reenters_alt_loc',
 'sType': 'second_order_false_belief',
 'qTypeRaw': 'first_order_1_tom',
 'qTypeTomOrNot': True,
 'qOrder': 'first_order',
 'storyHasToM': True,
 'answerMem': 'suitcase',
 'answerReal': 'bottle',
 'qToMandOmniReader': False,
 'answerMemOrReal': 'memory',
 'cands': '["suitcase", "bottle"]',
 'correct': 0,
 'falseTrueBelief': False,
 'factVsMind': 'mind'}

In [71]:
def process(instance):
    characters = find_characters(instance["story"])
    character_who_moved = find_character_who_moved(instance["story"])
    moved_object = find_moved_object(instance["story"])
    
    for c in nlp(instance["question"]).ents:
        if c.text != character_who_moved:
            answer = c.text
    
    distracting_characters = [c for c in characters if c not in [answer, character_who_moved]]
    
    cs = random.sample([character_who_moved, answer], 2)
    
    append_statement = "{} and {} plan to use the {} soon.".format(*cs, moved_object)
    
    question = "Who among the individuals would benefit from receiving helpful information?\n"
    for i, c in enumerate(random.sample(characters, len(characters))):
        question += f"{chr(65 + i)}. {c}\n"
    question += f"{chr(65 + len(characters))}. None of the above"

    return {
        "new_story": instance["story"] + " " + append_statement,
        "new_question": question,
        "new_answer": answer
    }
        

In [72]:
t = temp_subset.map(process, remove_columns = list(temp_subset.features))

Map:   0%|          | 0/564 [00:00<?, ? examples/s]

In [74]:
t[0]

{'new_story': 'Nathan entered the porch. Mia entered the porch. The radish is in the crate. Nathan exited the porch. Mia moved the radish to the drawer. Liam entered the front yard. Mia and Nathan plan to use the radish soon.',
 'new_question': 'Who among the individuals would benefit from receiving helpful information?\nA. Mia\nB. Nathan\nC. Liam\nD. None of the above',
 'new_answer': 'Nathan'}

In [70]:
list(temp_subset.features)

['story',
 'question',
 'answer',
 'i',
 'dataGenSeq',
 'sType',
 'qTypeRaw',
 'qTypeTomOrNot',
 'qOrder',
 'storyHasToM',
 'answerMem',
 'answerReal',
 'qToMandOmniReader',
 'answerMemOrReal',
 'cands',
 'correct',
 'falseTrueBelief',
 'factVsMind']